In [8]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import time
import os
from collections import Counter
import torch
import torch.nn as nn
from torch.optim.optimizer import Optimizer
from sklearn.model_selection import train_test_split

In [9]:
#pip install git+https://github.com/lukemelas/pytorch-pretrained-gans

In [10]:
#from pytorch_pretrained_gans import make_gan

In [11]:
#G = make_gan(gan_type='selfconditionedgan', model_name='self_conditioned')

In [12]:
#physical_devices = tf.config.list_physical_devices('GPU')
#print(physical_devices)
#for gpu in physical_devices:
    #tf.config.experimental.set_memory_growth(gpu, enable=True)

In [13]:
import os
print(os.listdir("../data"))

['ARXIV_V5_CHESTXRAY.pdf', 'BBox_List_2017.csv', 'Data_Entry_2017.csv', 'FAQ_CHESTXRAY.pdf', 'images_001', 'images_002', 'images_003', 'images_004', 'images_005', 'images_006', 'images_007', 'images_008', 'images_009', 'images_010', 'images_011', 'images_012', 'LOG_CHESTXRAY.pdf', 'README_CHESTXRAY.pdf', 'test_list.txt', 'train_val_list.txt']


In [14]:
xray_data = pd.read_csv('../data/Data_Entry_2017.csv')
xray_data.head(5)

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN


In [15]:
from glob import glob
#import os # already imported earlier

my_glob = glob('../data/images*/images/*.png')
print('Number of Observations: ', len(my_glob))

Number of Observations:  112120


In [16]:
full_img_paths = {os.path.basename(x): x for x in my_glob}
xray_data['full_path'] = xray_data['Image Index'].map(full_img_paths.get)

In [17]:
xray_data.head(5)

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,full_path
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN,../data\images_001\images\00000001_000.png
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN,../data\images_001\images\00000001_001.png
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN,../data\images_001\images\00000001_002.png
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN,../data\images_001\images\00000002_000.png
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN,../data\images_001\images\00000003_000.png


In [18]:
dummy_labels = ['Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema', 'Emphysema', 'Fibrosis', 'Effusion', 'Pneumonia', 'Pleural_Thickening', 
'Cardiomegaly', 'Nodule', 'Mass', 'Hernia']

for label in dummy_labels:
    xray_data[label] = xray_data['Finding Labels'].map(lambda result: 1.0 if label in result else 0)
display(xray_data)

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,...,Edema,Emphysema,Fibrosis,Effusion,Pneumonia,Pleural_Thickening,Cardiomegaly,Nodule,Mass,Hernia
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,30801,39,M,PA,2048,2500,0.168,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
112116,00030802_000.png,No Finding,0,30802,29,M,PA,2048,2500,0.168,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112117,00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112118,00030804_000.png,No Finding,0,30804,30,F,PA,2048,2500,0.168,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
def create_stratified_sample(df, sample_size=0.2, random_state=42):
    onehot_columns = df.columns[-14:]
    
    original_label_counts = df[onehot_columns].sum().to_dict()
    
    total_labels = sum(original_label_counts.values())
    original_label_freq = {k: v/total_labels for k, v in original_label_counts.items()}
    
    print("Original label distribution:")
    for label, count in original_label_counts.items():
        print(f"{label}: {count} ({original_label_freq[label]*100:.2f}%)")
    
    # Determine final sample size
    if isinstance(sample_size, float) and 0 < sample_size < 1:
        final_sample_size = int(len(df) * sample_size)
    elif isinstance(sample_size, int) and sample_size > 1:
        final_sample_size = min(sample_size, len(df))
    else:
        raise ValueError("sample_size must be a float between 0-1 or an integer > 1")
    
    label_rarity = {col: 1.0 / max(1, df[col].sum()) for col in onehot_columns}
    
    df['rarity_score'] = 0
    for col in onehot_columns:
        df.loc[df[col] == 1, 'rarity_score'] += label_rarity[col]

    rare_sample_size = int(final_sample_size * 0.2)
    rare_samples = df.sort_values('rarity_score', ascending=False).head(rare_sample_size)
    
    remaining_df = df[~df.index.isin(rare_samples.index)]
    
    remaining_df['label_combination'] = ''
    for col in onehot_columns:
        remaining_df['label_combination'] += remaining_df[col].astype(str)
    
    combination_counts = Counter(remaining_df['label_combination'])
    
    for idx, row in remaining_df.iterrows():
        if combination_counts[row['label_combination']] < 10:
            remaining_df.at[idx, 'label_combination'] = 'rare_combination'
    
    remaining_sample_size = final_sample_size - rare_sample_size
    
    if remaining_sample_size > 0:
        _, stratified_samples = train_test_split(
            remaining_df, 
            test_size=remaining_sample_size/len(remaining_df),
            stratify=remaining_df['label_combination'],
            random_state=random_state
        )
    else:
        stratified_samples = pd.DataFrame(columns=df.columns)
    
    sampled_df = pd.concat([rare_samples, stratified_samples])
    
    sampled_df = sampled_df.drop(['rarity_score', 'label_combination'], axis=1, errors='ignore')
    
    return sampled_df


In [20]:
sample_xray_data = xray_data[xray_data['Finding Labels']!= "No Finding"]
sample_xray_data = create_stratified_sample(sample_xray_data, sample_size=0.4)

Original label distribution:
Atelectasis: 11559.0 (14.24%)
Consolidation: 4667.0 (5.75%)
Infiltration: 19894.0 (24.51%)
Pneumothorax: 5302.0 (6.53%)
Edema: 2303.0 (2.84%)
Emphysema: 2516.0 (3.10%)
Fibrosis: 1686.0 (2.08%)
Effusion: 13317.0 (16.41%)
Pneumonia: 1431.0 (1.76%)
Pleural_Thickening: 3385.0 (4.17%)
Cardiomegaly: 2776.0 (3.42%)
Nodule: 6331.0 (7.80%)
Mass: 5782.0 (7.12%)
Hernia: 227.0 (0.28%)


C:\Temp\ipykernel_40416\3960046748.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rarity_score'] = 0
C:\Temp\ipykernel_40416\3960046748.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[8.65126741e-05 8.65126741e-05 8.65126741e-05 ... 8.65126741e-05
 8.65126741e-05 8.65126741e-05]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df[col] == 1, 'rarity_score'] += label_rarity[col]
C:\Temp\ipykernel_40416\3960046748.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [21]:
display(sample_xray_data)

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,...,Edema,Emphysema,Fibrosis,Effusion,Pneumonia,Pleural_Thickening,Cardiomegaly,Nodule,Mass,Hernia
63019,00015580_001.png,Cardiomegaly|Edema|Effusion|Fibrosis|Hernia,1,15580,78,F,PA,2992,2991,0.143,...,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
17611,00004747_000.png,Emphysema|Fibrosis|Hernia|Infiltration|Pleural...,0,4747,82,M,PA,2802,2991,0.143,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
77352,00018999_001.png,Emphysema|Fibrosis|Hernia|Mass,1,18999,64,M,PA,2992,2991,0.143,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
36018,00009507_001.png,Consolidation|Hernia|Infiltration|Pneumonia,1,9507,59,M,PA,2826,2905,0.143,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
36019,00009507_002.png,Effusion|Hernia|Pneumonia,2,9507,59,M,PA,2992,2991,0.143,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59281,00014663_035.png,Mass|Nodule,35,14663,63,M,AP,2500,2048,0.168,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
51914,00013111_047.png,Infiltration,47,13111,20,M,PA,2992,2991,0.143,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16284,00004344_015.png,Infiltration,15,4344,47,F,PA,2758,2991,0.143,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56128,00014004_009.png,Infiltration|Nodule|Pleural_Thickening,9,14004,58,F,PA,2400,2894,0.143,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [22]:
sample_xray_data['Finding Labels'].value_counts()

Finding Labels
Infiltration                               3321
Atelectasis                                1466
Effusion                                   1376
Nodule                                      941
Pneumothorax                                763
                                           ... 
Hernia|Infiltration|Pleural_Thickening        1
Cardiomegaly|Hernia|Mass                      1
Effusion|Hernia|Mass|Pleural_Thickening       1
Edema|Effusion|Hernia|Infiltration            1
Fibrosis|Hernia|Infiltration                  1
Name: count, Length: 822, dtype: int64

In [23]:
print(len(sample_xray_data))

20703


In [24]:
import cv2

In [25]:
def process_image(df,img_size):
    data = []
    full_paths = df['full_path']
    for index, img_path in enumerate(full_paths):
        if (index + 1) % 1000 == 0:
            print(index)
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        resized_arr = cv2.resize(image, (img_size, img_size))
        one_hot_labels = df.iloc[:, -14:].values
        data.append([resized_arr, one_hot_labels])
    return np.array(data)


In [26]:
img_data = process_image(sample_xray_data, 64)

999
1999
2999
3999
4999
5999


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\alloc.cpp:73: error: (-4:Insufficient memory) Failed to allocate 1048576 bytes in function 'cv::OutOfMemoryError'


In [ ]:
class Acgan:
    def __init__(self, eta, batch_size, epochs, weight_decay, latent_space,
                 image_shape, kernel_size):
        self.eta = eta
        self.batch_size = batch_size
        self.epochs = epochs
        self.weight_decay = weight_decay
        self.latent_space = latent_space
        self.image_shape = image_shape
        self.kernel_size = kernel_size
    def data(self, images, labels):
        ytrain = tf.keras.utils.to_categorical(labels)
        self.images = images
        self.labels = ytrain
    def samples(self, G, noize, labels):
        images = G.predict([noize, labels])
        ys = np.argmax(labels, axis = 1)
        plt.figure(figsize = (12, 4))
        for i in range(16):
            plt.subplot(2, 8, (i + 1))
            plt.imshow(images[i], cmap = 'gray')
            plt.title(ys[i])
        plt.show()
    def generator(self, inputs, labels):
        filters = [256, 128, 64, 32]
        padding = 'same'
        x = inputs
        y = labels
        x = layers.concatenate([x, y])
        x = layers.Dense(1024, )(x)
        x = layers.Dense(8*8*filters[0],
                         kernel_regularizer = tf.keras.regularizers.L2(0.001))(x)
        x = layers.Reshape((8, 8, filters[0]))(x)
        for filter in filters:
            if filter >= 64:
                strides = 2
            else:
                strides = 1
            x = LayerNormalization()(x)
            x = layers.Activation('relu')(x)
            x = Conv2DTranspose(filter, kernel_size = self.kernel_size, padding = padding,
                      strides = strides)(x)
        x = Conv2DTranspose(3, kernel_size = self.kernel_size, padding = padding)(x)
        x = layers.Activation('sigmoid')(x)
        self.generatorModel = models.Model(inputs = [inputs, labels],
                                           outputs = x,
                                           name = 'generator')
    def discriminator(self, inputs):
        x = inputs
        filters = [32, 64, 128, 256]
        padding = 'same'
        for filter in filters:
            if filter < 256:
                strides = 2
            else:
                strides = 1
            x = Conv2D(filter, kernel_size = self.kernel_size, padding = padding,
                      strides = strides,
                      kernel_regularizer = tf.keras.regularizers.L2(0.001))(x)
            x = LeakyReLU(alpha = 0.2)(x)
        x = layers.Flatten()(x)
        outputs = Dense(1, )(x)
        labelsOutput = Dense(256,
                             kernel_regularizer = tf.keras.regularizers.L2(0.001))(x)
        labelsOutput = Dropout(0.3)(labelsOutput)
        labelsOutput = Dense(2,)(labelsOutput)
        labelsOutput = layers.Activation('softmax')(labelsOutput)
        self.discriminatorModel = models.Model(inputs = inputs,
                                               outputs = [outputs, labelsOutput],
                                               name = 'discriminator')
    def build(self,):
        generatorInput = layers.Input(shape = (self.latent_space))
        discriminatorInput = layers.Input(shape = (self.image_shape))
        labelsInput = layers.Input(shape = (2, ))
        self.generator(generatorInput, labelsInput)
        self.discriminator(discriminatorInput)
        G = self.generatorModel
        D = self.discriminatorModel
        D.compile(loss = ['mse', 'binary_crossentropy'],
                 optimizer = tf.keras.optimizers.RMSprop(learning_rate = self.eta,
                                                        weight_decay = self.weight_decay))
        D.summary()
        G.summary()
        D.trainable = False
        GAN = models.Model(inputs = [generatorInput, labelsInput],
                           outputs = D(G([generatorInput, labelsInput])))
        GAN.compile(loss = ['mse', 'binary_crossentropy'],
                   optimizer = tf.keras.optimizers.RMSprop(learning_rate = self.eta*0.5,
                                                          weight_decay = self.weight_decay*0.5))
        GAN.summary()
        return G, D, GAN
    def trainAlgorithm(self, G, D, GAN):
        for epoch in range(self.epochs):
            indexs = np.random.randint(0, len(self.images), size = (self.batch_size, ))
            realImages = self.images[indexs]
            realLabels = self.labels[indexs]
            realTag = tf.ones(shape = (self.batch_size, ))
            noize = tf.random.uniform(shape = (self.batch_size,
                                              self.latent_space), minval = -1,
                                     maxval = 1)
            fakeLabels = tf.keras.utils.to_categorical(np.random.choice(range(2), size = (self.batch_size)),
                                                      num_classes = 2)
            fakeImages = tf.squeeze(G.predict([noize, fakeLabels], verbose = 0))
            fakeTag = tf.zeros(shape = (self.batch_size, ))
            allImages = np.vstack([realImages, fakeImages])
            allLabels = np.vstack([realLabels, fakeLabels])
            allTags = np.hstack([realTag, fakeTag])
            _, dlossTag, dlossLabels = D.train_on_batch(allImages, [allTags, allLabels])
            noize = tf.random.uniform(shape = (self.batch_size,
                                              self.latent_space), minval = -1,
                                     maxval = 1)
            _, glossTag, glossLabels = GAN.train_on_batch([noize, fakeLabels], [realTag, fakeLabels])
            if epoch % 5000 == 0:
                print('Epoch: {}'.format(epoch))
                print('discriminator loss: [tag: {}, labels: {}], generator loss: [tag: {}, labels: {}]'.format(dlossTag,
                                                                                                                dlossLabels,
                                                                                                                glossTag,
                                                                                                                glossLabels))
                self.samples(G, noize, fakeLabels)